Shawn (HyungJoon) Yoon

I followed the honor code.

10hrs

In [11]:
import pandas as pd
from linearmodels import OLS
import numpy as np

In [12]:
df_data = pd.read_csv("CerealData.csv", index_col=0)
df_data.head()

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar
j,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0
2,General Mills,Cheerios,3.16,4.38,0,0,110,2.0,1.0
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0


In [13]:
########## 1A

covariates = ['Price', 'Adult', 'Kids', 'Calories', 'Fat', 'Sugar']

s0 = 100 - df_data.MarketShare.sum()
df_data['log_ratio_market_shares'] = np.log(df_data.MarketShare / s0)
df_data.head()

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares
j,,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0,-1.452402
2,General Mills,Cheerios,3.16,4.38,0,0,110,2.0,1.0,-1.710543
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0,-1.791347
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0,-1.847341
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0,-2.183290


In [14]:
# Run regression
formula_str = "log_ratio_market_shares ~ 1 + Price + Adult + Kids + Calories + Fat + Sugar"# complete the formula
model = OLS.from_formula(formula_str, df_data)# complete

res = model.fit()
print(res.summary)
beta_price = res.params.Price

                               OLS Estimation Summary                              
Dep. Variable:     log_ratio_market_shares   R-squared:                      0.3486
Estimator:                             OLS   Adj. R-squared:                 0.2577
No. Observations:                       50   F-statistic:                    20.666
Date:                     Mon, Oct 28 2019   P-value (F-stat)                0.0021
Time:                             06:17:42   Distribution:                  chi2(6)
Cov. Estimator:                     robust                                         
                                                                                   
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -2.2186     0.6094    -3.6407     0.0003     -3.4130     -1.0242
Price       

In [16]:
df_data.head()

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares
j,,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0,-1.452402
2,General Mills,Cheerios,3.16,4.38,0,0,110,2.0,1.0,-1.710543
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0,-1.791347
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0,-1.847341
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0,-2.183290


In [52]:
########## 1D 
# Create vector of own-price demand elasticities
eta = df_data['Price']*beta_price*(1-(df_data.MarketShare/100))
# Complete the formula
eta.mean()

-0.2364678342125601

It seems inelastic.

In [21]:
########## 1E
# Create the eV for each j, i.e. exp(V_j). Notice that exp(V_j) = sj/s0
df_data['eV'] = df_data.MarketShare / s0
df_data.head()

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares,eV
j,,,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0,-1.452402,0.234007
2,General Mills,Cheerios,3.16,4.38,0,0,110,2.0,1.0,-1.710543,0.180768
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0,-1.791347,0.166735
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0,-1.847341,0.157656
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0,-2.183290,0.112670


In [25]:
# Create the eV for Kellogg's Raisin Bran
eV_KRB = df_data.eV.loc[5]


In [26]:
########## 1F
sj_NoKRB = 100 * df_data.eV / (1+ sum(df_data.eV) - eV_KRB ) 
sj_NoKRB.sum()

77.89657653952914

In [29]:
df_data['MarketShare'].sum()

75.77000000000001

In [30]:
df_data['ratio'] = sj_NoKRB / df_data.MarketShare
print(df_data.ratio.describe())

# Check: the new market shares should sum to 100, after including additional substitution to the outside option.
sum(sj_NoKRB)-sj_NoKRB[5] + np.mean(df_data.ratio) * s0 

count    5.000000e+01
mean     1.028066e+00
std      2.436510e-16
min      1.028066e+00
25%      1.028066e+00
50%      1.028066e+00
75%      1.028066e+00
max      1.028066e+00
Name: ratio, dtype: float64


99.99999999999999

In [33]:
sj_NoKRB.head()

j
1    5.829135
2    4.502930
3    4.153387
4    3.927213
5    2.806621
Name: eV, dtype: float64

In [34]:
df_data.head()

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares,eV,ratio
j,,,,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0,-1.452402,0.234007,1.028066
2,General Mills,Cheerios,3.16,4.38,0,0,110,2.0,1.0,-1.710543,0.180768,1.028066
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0,-1.791347,0.166735,1.028066
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0,-1.847341,0.157656,1.028066
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0,-2.183290,0.112670,1.028066


In [38]:
# finding Post Raisin Bran
df_data[df_data.Manufacturer=='Post']

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares,eV,ratio
j,,,,,,,,,,,,
9,Post,Grape Nuts,2.14,2.12,1,0,200,1.0,7.0,-2.436175,0.087495,1.028066
16,Post,Raisin Bran,2.23,1.46,0,0,190,1.0,20.0,-2.809155,0.060256,1.028066
34,Post,Honey Bunches of Oats,2.85,0.95,1,0,125,2.2,6.0,-3.238885,0.039208,1.028066
35,Post,Great Grains,2.90,0.89,1,0,215,5.5,10.5,-3.304125,0.036731,1.028066
44,Post,Fruity Pebbles,3.32,0.83,0,1,110,1.0,12.0,-3.373921,0.034255,1.028066
49,Post,Honeycomb,3.40,0.74,0,1,110,0.0,11.0,-3.488697,0.030541,1.028066


In [39]:
# finding Kellogg's Corn Pops
df_data[df_data.Manufacturer=="Kellogg's"]

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares,eV,ratio
j,,,,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0,-1.452402,0.234007,1.028066
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0,-1.791347,0.166735,1.028066
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0,-1.847341,0.157656,1.028066
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0,-2.183290,0.112670,1.028066
8,Kellogg's,Special K,3.48,2.16,1,0,110,0.0,3.0,-2.417483,0.089146,1.028066
12,Kellogg's,Frosted Mini Wheats,2.62,1.84,1,0,170,1.0,10.0,-2.577826,0.075939,1.028066
13,Kellogg's,NutriGrain,2.87,1.55,1,0,100,1.0,0.0,-2.749337,0.063970,1.028066
14,Kellogg's,Mueslix,3.31,1.53,1,0,200,4.0,13.0,-2.762324,0.063145,1.028066
18,Kellogg's,Corn Pops,3.51,1.46,0,0,120,0.0,14.0,-2.809155,0.060256,1.028066


H. It's unrealistic, because every top 50 brands' market share increased with equal ratio. Mathematically, it makes sense, but in real terms, different factors will affect customers to go for different brands. 

The pool has gotten smaller. Therefore, it seems that the market share has risen, but it didn't really affect the increased in quantity demand. 

I. Assume that every cereal brand j sets prices to maximize its own profits. Write the profit
maximization problem for brand j, assuming constant marginal cost c j.

MAX$P_j$ $\pi$ = $S_j$($P_j$)($P_j$ - $C_j$)

J) Using the first-order condition, find the equation for the marginal cost c j that would make the
observed prices profit-maximizing. This c j should be a function of β price, market share s j, and
price p j.

MAX$p_j$  {$S_j$($p_j$) * ($p_j$ - $c_j$)}

FOC

$S_j$($p_j$) +  (∂$S_j$/∂$p_j$)($p_j$ - $c_j$) = 0

(∂$S_j$/∂$p_j$) = $\beta_{price}$ * $S_j$(1-$S_j$)

b/c $S_j$ = exp($V_j$)/(exp($V_0$) + $\Sigma$ (exp($V_k$))

$S_j$($p_j$) +  ($\beta_{price}$ * $S_j$(1-$S_j$))($p_j$ - $c_j$) = 0

Solve for $c_j$ 

$c_j$ = $p_j$ + $S_j$/($\beta_{price}$ * $S_j$(1-$S_j$))

K) Using R, construct the vector of marginal costs for each brand j. Report the minimum,
maximum, and mean.

In [45]:
c_j = df_data['Price'] + (df_data['MarketShare']/100)/(beta_price*(df_data['MarketShare']/100)*(1-(df_data['MarketShare']/100)))
print(c_j.describe())

count    50.000000
mean     -9.646590
std       0.548273
min     -11.376702
25%      -9.971412
50%      -9.569460
75%      -9.225750
max      -8.621183
dtype: float64


L) Explain why your answers to K are negative. Hint: this is driven by your answer to question D.

In [50]:
df_data.head()

,Manufacturer,Brand,Price,MarketShare,Adult,Kids,Calories,Fat,Sugar,log_ratio_market_shares,eV,ratio
j,,,,,,,,,,,,
1,Kellogg's,Corn Flakes,1.81,5.67,0,0,100,0.0,2.0,-1.452402,0.234007,1.028066
2,General Mills,Cheerios,3.16,4.38,0,0,110,2.0,1.0,-1.710543,0.180768,1.028066
3,Kellogg's,Rice Krispies,2.96,4.04,0,0,120,0.0,3.0,-1.791347,0.166735,1.028066
4,Kellogg's,Frosted Flakes,2.52,3.82,0,0,120,0.0,13.0,-1.847341,0.157656,1.028066
5,Kellogg's,Raisin Bran,2.34,2.73,0,0,200,1.5,18.0,-2.183290,0.112670,1.028066


In [49]:
beta_price

-0.0803922114503628

In [48]:
########## 1L
cj = df_data['Price'] + (df_data['MarketShare']/100)/(beta_price*(df_data['MarketShare']/100)*(1-(df_data['MarketShare']/100)))
# complete formula 

print(cj.describe())

# Check: make sure markups all positive
markup=(df_data.Price-cj)# complete 
print(markup)

count    50.000000
mean     -9.646590
std       0.548273
min     -11.376702
25%      -9.971412
50%      -9.569460
75%      -9.225750
max      -8.621183
dtype: float64
j
1     13.186702
2     13.008801
3     12.962709
4     12.933059
5     12.788132
6     12.739672
7     12.726638
8     12.713630
9     12.708435
10    12.703243
11    12.687695
12    12.672184
13    12.634856
14    12.632290
15    12.631007
16    12.623316
17    12.597748
18    12.623316
19    12.609241
20    12.604130
21    12.600300
22    12.595196
23    12.593921
24    12.590097
25    12.586275
26    12.581183
27    12.579911
28    12.579911
29    12.574824
30    12.574824
31    12.572282
32    12.572282
33    12.563394
34    12.558320
35    12.550717
36    12.560856
37    12.557052
38    12.550717
39    12.550717
40    12.550717
41    12.544389
42    12.543124
43    12.539331
44    12.543124
45    12.536803
46    12.534276
47    12.530489
48    12.533014
49    12.531751
50    12.522920
dtype: float64


The firms on average have inelastic elasticity. This makes sense graphically but it doesn't make sense mathmatically. 

The firms are operating at the very right side of the demand curve. Because that's where MR=MC. 

However, mathematically, p = (mc/(1+$\eta$)) doesn't hold in anyway possible. 

M) In reality, every cereal manufacturer should set the prices of all of its brands jointly to maximize
total profits. For example, Kellogg’s should set the prices of Corn Flakes, Frosted Flakes, Raisin
Bran, Special K, and all other Kellogg’s brands together to maximize the sum of profits. Write the
profit function and briefly discuss the intuition for what each manufacturer needs to consider
when it set prices for all of its brands.

N) Will prices be higher or lower when manufacturers set prices for all their brands jointly (as in
part M) compared to when manufacturers set prices for each brand separately (as in J)? Why?